# TD Learning Using Neural Networks

In this notebook, we will perform TD Learning on a one dimensional maze very similarly to the previous TD Learning notebook. However, we will no longer be using a table. Rather, we will replace this table with a neural network. Since neural networks can approximate functions, we will use one to approximate the table that would be used otherwise. With small, simple mazes, the benefits may not be immediately recognizable. However with more complex scenarios, this can save a great amount of space. There is an added benefit of not having to deal with complex tables.

We will be using _Holographic Reduced Representations_ to encode information for the neural network. This is able to represent the states of the maze, as well as things such as colored signals, and allow them to be combined in 